In [ ]:
import math
from abc import ABC, abstractmethod
from collections import defaultdict

class MCTS(ABC):
    # 初始化MCTS类，设置探索权重
    def __init__(self, exploration_weight=1):
        self.Q = defaultdict(float)  # 存储节点的总奖励
        self.N = defaultdict(int)  # 存储节点的访问次数
        self.children = dict()  # 存储节点的子节点
        self.exploration_weight = exploration_weight  # 探索权重

    # 搜索函数，从根节点开始，选择没有被探索过的节点
    def _search(self, node):
        # 定义一个空列表path，用于存储搜索路径
        path = []
        # 当node在children中且children[node]不为空时，执行循环
        while True:
            # 将node添加到path中
            path.append(node)
            # 如果node不在children中或者children[node]为空，则返回path
            if node not in self.children or not self.children[node]:
                return path
            # 获取node的未探索子节点
            unexplored = self.children[node] - self.children.keys()

            # 如果unexplored不为空，则从unexplored中弹出一个节点n，并将n添加到path中，然后返回path
            if unexplored:
                n = unexplored.pop()
                path.append(n)
                return path
            
            # 否则，将node设置为uct_select(node)的返回值
            node = self._uct_select(node)
            
    
    # 定义一个_expend函数，用于展开节点
    def _expend(self, node):
        # 如果节点已经在子节点中，则返回
        if node in self.children:
            return
        # 否则，将节点添加到子节点中，并调用find_children函数获取子节点
        self.children[node] = node.find_children()
        
    # 定义_rollout函数，用于模拟从当前节点开始的一轮游戏
    def _rollout(self, node):
        # 当节点不是终止节点时，继续循环
        while True:
            # 如果节点是终止节点，则返回节点的奖励
            if node.is_terminal():
                return node.reward
            # 否则，随机选择一个子节点
            node = node.find_randon_child()    
            
    def _backpropagate(self, path, reward):
        # 遍历路径中的每个节点
        for node in reversed(path):
            self.N[node] += 1
            self.Q[node] += reward
            
    # 执行迭代
    def do_iteration(self, node):
        # 搜索路径
        path = self._search(node)
        # 获取路径的最后一个节点
        leaf = path[-1]
        # 扩展叶子节点
        self._expend(leaf)
        # 对叶子节点进行回溯
        # 回溯路径
        reward = self._rollout(leaf)
        self._backpropagate(path, reward)
        
    def choose(self,node):
        # 如果node不在self.children中，则返回node的随机子节点
        if node not in self.children:
            return node.find_random_child()
        
        # 定义一个函数score，用于计算节点的得分
        def score(n):
            # 如果节点的访问次数为0，则返回负无穷大
            if self.N[n] == 0:
                return float('-inf')
            # 否则返回节点的Q值除以访问次数
            return self.Q[n] / self.N[n]
        
        # 返回得分最高的子节点
        return max(self.children[node], key=score)
    
    
    # UCT选择函数，如果节点都被探索过了，通过该函数选择最优节点
    def _uct_select(self, node):

        assert all(n in self.children for n in self.children[node]) # 确保所有子节点都被探索过

        log_N_vertex = math.log(self.N[node])

        def uct(n):
            return self.Q[n] / self.N[n] + self.exploration_weight * math.sqrt(log_N_vertex / self.N[n])

        return max(self.children[node], key=uct)

In [ ]:
class Node(ABC):
    
    # 定义一个抽象方法，用于查找子节点
    @abstractmethod
    def find_children(self):
        
        pass
    
    # 定义一个抽象方法，用于随机查找子节点
    @abstractmethod 
    def find_random_child(self):
        
        pass

    # 定义一个抽象方法，用于判断节点是否为终端节点
    @abstractmethod
    def is_terminal(self): 

        pass

    # 定义一个抽象方法，用于返回节点的奖励值
    @abstractmethod
    def reward(self):

        pass
    
    # 定义一个抽象方法，用于返回节点的哈希值
    @abstractmethod
    def __hash__(self):

        pass
    
    @abstractmethod
    def __eq__(self, node1, node2):

        '''
        判断两个对象是否相等
        :param other: 另一个对象
        :return: 如果两个对象相等，返回True，否则返回False
        '''
        pass
 